In [ ]:
# Imports
import os
import sys

# Local Imports
sys.path.insert(0, 'py')
from encode_text_table import encode_text_table
from google_cloud_platform_ocr import google_cloud_platform_ocr
from json_to_text_table import json_to_text_table
from process_page_image import process_page_image
from document_pdf_to_page_jpegs import document_pdf_to_page_jpegs
from segment_entries import segment_entries
from text_table_to_entries import text_table_to_entries

# Directory and file definitions
raw_columns_image_dir = 'image/columns/'
raw_page_image_dir = 'image/raw_jpg/'
raw_pdf_image_dir = 'image/raw_pdf/'
processed_image_dir = 'image/processed_jpg/'
txt_dir = 'txt/'

# Google cloud platform API key
api_key = '*****'

In [ ]:
# Name of PDF file without .pdf extension
filename_base = 'Gray-PDF-Duplex2'

In [ ]:
# Extend relative paths to absolute paths
cwd = os.getcwd()
raw_page_image_dir = cwd + '/' + raw_page_image_dir
raw_pdf_image_dir = cwd + '/' + raw_pdf_image_dir
raw_columns_image_dir = cwd + '/' + raw_columns_image_dir
processed_image_dir = cwd + '/' + processed_image_dir
txt_dir = cwd + '/' + txt_dir

# Define filenames
pdf_filename = filename_base + '.pdf'
txt_file = filename_base + '.txt'
output_filename = filename_base + '_'
raw_pdf_image_file = raw_pdf_image_dir + pdf_filename

In [ ]:
# Segment document PDF into page JPGs
doc_pdf_to_page_jpgs = document_pdf_to_page_jpegs(raw_pdf_image_file, raw_page_image_dir, output_filename, '600dpi')
doc_pdf_to_page_jpgs.run()

In [ ]:
# 
gcp_ocr = google_cloud_platform_ocr(api_key)
pr_pg_im = process_page_image(processed_image_dir, 2, '600dpi')

# Process page JPGs
for page in range(10):
    
    page_num = page + 1
    
    print(page_num)

    filename = output_filename + str(page_num) + '.jpg'
    raw_page_image_file = raw_page_image_dir + filename
    
    segment_map = pr_pg_im.run(raw_page_image_file)
                                                 
    if True:
        processed_segments_dir = processed_image_dir + 'segments_tmp/'
        processed_segment_files = os.listdir(processed_segments_dir)
        for filename in processed_segment_files:
            gcp_ocr.do_ocr(processed_segments_dir + filename)

        json_files = os.listdir('json/')
        for i in range(len(segment_map)):
            text_table = []
            for j in range(segment_map[i][2]-segment_map[i][1]):
                filename = 'segment' + str(segment_map[i][1] + j) + '.json'
                text_table_tmp = json_to_text_table().run('json/' + filename)
                text_table.append(encode_text_table().run(text_table_tmp))
            entries = text_table_to_entries().run(text_table)
            seg_entries = segment_entries(txt_dir, txt_file)
            seg_entries.run(pdf_filename, page_num, entries)